# Video Watch Percentage

This notebook loads csv file that contains user activities on a specified course, then calculates the time spent by each user on each video.

## 1. Define function to calculate duration

In [1]:
#multiple start time - last function that works
import pandas as pd
from datetime import datetime
import re

def construct_usr_component_record(input_usr_dict):
    # Define pattern to find component id and type
    pattern = r'(?:/courses/[^/]+/(discussion)/[^/]+/([\da-f]+)/)|(?:@(\w+)\+block@([\da-f]+)/)'

    # Define initial state
    prev_seq_id = "nan"
    prev_comp_id = "nan"
    cur_comp_type = "nan"
    tmp_usr_component_sequence = dict()
    cur_s_time = "nan"
    
    # Loop through each activity
    for row_idx, (tmp_type, tmp_source, tmp_time) in enumerate(zip(input_usr_dict.event_type, input_usr_dict.event_source, input_usr_dict.time)):
        
        # Filter only non-server sources
        if tmp_source != "server":
            continue

        # Find all matches in the string
        matches = re.findall(pattern, tmp_type)
        
        # Check if there are any matches (only sequences and components)
        if matches:
            for match in matches:
                if match[0] and match[1]:  # Matched discussion                    
                    X, Y = match[0], match[1]
                elif match[2] and match[3]:  # Matched component and sequential
                    X, Y = match[2], match[3]

                if X == "sequential":  # Sequential 
                    if prev_seq_id != Y:
                        prev_seq_id = Y
                else:  # html, problem, video components
                    if prev_comp_id != Y:
                        # Record the start time for a new component
                        cur_comp_type = X
                        cur_s_time = tmp_time

                        if prev_comp_id != "nan": 
                            prev_e_time = tmp_time
                            
                            # Convert string times to datetime objects
                            prev_s_time_dt = datetime.strptime(prev_s_time, "%Y-%m-%dT%H:%M:%S.%f%z")
                            prev_e_time_dt = datetime.strptime(prev_e_time, "%Y-%m-%dT%H:%M:%S.%f%z")
                            
                            # Calculate the duration in minutes
                            duration = (prev_e_time_dt - prev_s_time_dt).total_seconds() / 60.0

                            if duration > 60:
                                continue

                            # Append the new times and duration
                            if prev_comp_id in tmp_usr_component_sequence:
                                tmp_usr_component_sequence[prev_comp_id]["s_time"].append(prev_s_time)
                                tmp_usr_component_sequence[prev_comp_id]["e_time"].append(prev_e_time)
                                tmp_usr_component_sequence[prev_comp_id]["duration"].append(duration)
                            else:
                                tmp_usr_component_sequence[prev_comp_id] = {
                                    "type": prev_comp_type,
                                    "id": prev_comp_id,
                                    "s_time": [prev_s_time],
                                    "e_time": [prev_e_time],
                                    "duration": [duration]
                                }

                        # Update to the current component and start time
                        prev_comp_id = Y
                        prev_s_time = cur_s_time
                        prev_comp_type = cur_comp_type

    # Convert to DataFrame
    return pd.DataFrame(tmp_usr_component_sequence)

## 2. Initialize paths and file names

In [ ]:
COURSE_NAME = 'ARCH301'
COURSE_YEAR = '2021'
EVENT_LOG_FILE_PATH = 'course_data/'+ COURSE_NAME + '/categorized/categorized_student_event_' + COURSE_YEAR+ '_csv_data.csv'
ENROLLMENT_MODE = 'verified'
OUTLIERS_FILE_PATH = 'course_data/'+ COURSE_NAME + '/outliers.json'
VIDEO_DURATION_FILE_PATH = 'course_data/'+ COURSE_NAME + '/'+COURSE_NAME + '_video_duration.xlsx'
WATCH_PERCENTAGE_FILE_PATH = 'course_data/'+ COURSE_NAME + '/' + '/watch_percentage/'+ ENROLLMENT_MODE+'/' +COURSE_NAME + '_' + ENROLLMENT_MODE + '_watch_percentage.xlsx'

## 3. Load data

In [3]:
df = pd.read_csv(EVENT_LOG_FILE_PATH)

# Filter only `audit` users
df_filtered = df[df["enrollment_mode"] == ENROLLMENT_MODE]

# Keep only relevant columns
df_filtered = df_filtered[["username", "event_type", "event_source", "time"]]


## 4. Exclude outlier users

In [4]:
import json

with open(OUTLIERS_FILE_PATH, 'r') as f:
    outliers = json.load(f)

df_filtered_wo_outliers = df_filtered[~df_filtered['username'].isin(outliers)]
df_filtered_wo_outliers

,username,event_type,event_source,time
598,0f37ae4f9d085c66b68820fa29f8bc7577272c39,edx.course.enrollment.activated,server,2021-12-22T13:44:06.760319+00:00
599,0f37ae4f9d085c66b68820fa29f8bc7577272c39,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-22T13:45:01.045811+00:00
600,0f37ae4f9d085c66b68820fa29f8bc7577272c39,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-22T13:45:01.396818+00:00
601,0f37ae4f9d085c66b68820fa29f8bc7577272c39,/api/discounts/course/course-v1:TokyoTechX+ARC...,server,2021-12-22T13:45:16.445185+00:00
3581,4848720da58b52312b0340e836eb8e928474aa8d,edx.course.enrollment.activated,server,2021-12-27T05:48:11.885338+00:00
...,...,...,...,...
14735,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-27T15:05:26.412350+00:00
14736,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-27T15:06:13.147438+00:00
14737,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-27T15:06:21.079184+00:00
14738,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,/courses/course-v1:TokyoTechX+ARCH301x+3T2021a...,server,2021-12-27T15:06:30.588650+00:00


## 5. Calculate duration of engagement on each content

In [5]:
# Function to apply processing per user
def process_user_logs(user_df):
    return construct_usr_component_record(user_df)

# Group by username and process each user's data
results = df_filtered_wo_outliers.groupby("username").apply(process_user_logs)

# Display the results
results.head(5)

/var/folders/hq/mr_d9f8x11dfzc_f03z9xzbw0000gn/T/ipykernel_53311/3451499937.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df_filtered_wo_outliers.groupby("username").apply(process_user_logs)


2631b553806d454f811642e76270c83a  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                    html   
                                         id          2631b553806d454f811642e76270c83a   
                                         s_time    [2021-12-27T05:51:33.473217+00:00]   
                                         e_time    [2021-12-27T06:48:10.877021+00:00]   
                                         duration                 [56.62339673333334]   

                                                     edbac534322343218b7e4e4ab9960bef  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                    html   
                                         id          edbac534322343218b7e4e4ab9960bef   
                                         s_time    [2021-12-27T06:48:10.877021+00:00]   
                                         e_time    [2021-12-27T07:13:01.989851+00:00]   
                                         duration                        [24.8518805]   

                                                     768f8fd6673344d78420a99c38328bc6  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                    html   
                                         id          768f8fd6673344d78420a99c38328bc6   
                                         s_time    [2021-12-27T07:13:01.989851+00:00]   
                                         e_time    [2021-12-27T07:14:42.288207+00:00]   
                                         duration                [1.6716392666666666]   

                                                   5bafd48096b0125ea97b7f1020b1a4e113be8e20  \
username                                                                                      
4848720da58b52312b0340e836eb8e928474aa8d type                                    discussion   
                                         id        5bafd48096b0125ea97b7f1020b1a4e113be8e20   
                                         s_time          [2021-12-27T07:14:42.288207+00:00]   
                                         e_time          [2021-12-27T07:15:25.981863+00:00]   
                                         duration                               [0.7282276]   

                                                     c3403b4bddfb40759d6dc0d630a75484  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                   video   
                                         id          c3403b4bddfb40759d6dc0d630a75484   
                                         s_time    [2021-12-27T07:15:25.981863+00:00]   
                                         e_time    [2021-12-27T07:16:19.067708+00:00]   
                                         duration                [0.8847640833333333]   

                                                     eca85b3f2d39415d86cc7aa1130cf96b  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                   video   
                                         id          eca85b3f2d39415d86cc7aa1130cf96b   
                                         s_time    [2021-12-27T07:16:19.067708+00:00]   
                                         e_time    [2021-12-27T07:20:05.605708+00:00]   
                                         duration                [3.7756333333333334]   

                                                     24c791181d8145889626b1b210456726  \
username                                                                                
4848720da58b52312b0340e836eb8e928474aa8d type                                 problem   


In [6]:
# Resetting the index to get 'username' as a column, if it's currently set as an index
result_reset = results.reset_index()

# Melt the dataframe to restructure it
result_melted = result_reset.melt(id_vars=['username', 'level_1'], 
                                  value_vars=result_reset.columns[2:],  # all ID columns
                                  var_name='id', value_name='value')
result_pivoted = result_melted.pivot_table(index=['username', 'id'], columns='level_1', values='value', aggfunc='first')
# Rename existing 'id' column to avoid conflict
result_pivoted = result_pivoted.rename(columns={'id': 'id_old'})

# Reset index, which adds the index 'id' column back
result_pivoted = result_pivoted.reset_index()

In [7]:
result_pivoted.drop(columns=['id_old'], inplace=True)
result_pivoted

level_1,username,id,duration,e_time,s_time,type
0,4848720da58b52312b0340e836eb8e928474aa8d,24c791181d8145889626b1b210456726,[0.14779050000000002],[2021-12-27T07:20:14.473138+00:00],[2021-12-27T07:20:05.605708+00:00],problem
1,4848720da58b52312b0340e836eb8e928474aa8d,2631b553806d454f811642e76270c83a,[56.62339673333334],[2021-12-27T06:48:10.877021+00:00],[2021-12-27T05:51:33.473217+00:00],html
2,4848720da58b52312b0340e836eb8e928474aa8d,5bafd48096b0125ea97b7f1020b1a4e113be8e20,[0.7282276],[2021-12-27T07:15:25.981863+00:00],[2021-12-27T07:14:42.288207+00:00],discussion
3,4848720da58b52312b0340e836eb8e928474aa8d,768f8fd6673344d78420a99c38328bc6,[1.6716392666666666],[2021-12-27T07:14:42.288207+00:00],[2021-12-27T07:13:01.989851+00:00],html
4,4848720da58b52312b0340e836eb8e928474aa8d,c3403b4bddfb40759d6dc0d630a75484,[0.8847640833333333],[2021-12-27T07:16:19.067708+00:00],[2021-12-27T07:15:25.981863+00:00],video
5,4848720da58b52312b0340e836eb8e928474aa8d,eca85b3f2d39415d86cc7aa1130cf96b,[3.7756333333333334],[2021-12-27T07:20:05.605708+00:00],[2021-12-27T07:16:19.067708+00:00],video
6,4848720da58b52312b0340e836eb8e928474aa8d,edbac534322343218b7e4e4ab9960bef,[24.8518805],[2021-12-27T07:13:01.989851+00:00],[2021-12-27T06:48:10.877021+00:00],html
7,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,2631b553806d454f811642e76270c83a,[4.1803520999999995],[2021-12-23T13:24:06.288436+00:00],[2021-12-23T13:19:55.467310+00:00],html
8,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,768f8fd6673344d78420a99c38328bc6,[5.1597217833333335],[2021-12-23T13:19:55.467310+00:00],[2021-12-23T13:14:45.884003+00:00],html
9,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,a511e497e3a14e50b7e44aca1f726ebe,[14.019456566666666],[2021-12-23T13:14:45.884003+00:00],[2021-12-23T13:00:44.716609+00:00],video


## 6. Filter only video content

In [8]:
result_pivoted = result_pivoted[result_pivoted['type'] == 'video']

result_pivoted.head(10)

level_1,username,id,duration,e_time,s_time,type
4,4848720da58b52312b0340e836eb8e928474aa8d,c3403b4bddfb40759d6dc0d630a75484,[0.8847640833333333],[2021-12-27T07:16:19.067708+00:00],[2021-12-27T07:15:25.981863+00:00],video
5,4848720da58b52312b0340e836eb8e928474aa8d,eca85b3f2d39415d86cc7aa1130cf96b,[3.7756333333333334],[2021-12-27T07:20:05.605708+00:00],[2021-12-27T07:16:19.067708+00:00],video
9,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,a511e497e3a14e50b7e44aca1f726ebe,[14.019456566666666],[2021-12-23T13:14:45.884003+00:00],[2021-12-23T13:00:44.716609+00:00],video
10,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,c3403b4bddfb40759d6dc0d630a75484,[1.58707355],[2021-12-23T12:53:54.625610+00:00],[2021-12-23T12:52:19.401197+00:00],video
11,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,eca85b3f2d39415d86cc7aa1130cf96b,"[6.834849983333333, 5.904826283333334]","[2021-12-23T13:00:44.716609+00:00, 2021-12-23T...","[2021-12-23T12:53:54.625610+00:00, 2021-12-23T...",video


## 7. Obtain total video watch duration

In [9]:
result_pivoted['total_watch_duration'] = result_pivoted['duration'].apply(sum)

/var/folders/hq/mr_d9f8x11dfzc_f03z9xzbw0000gn/T/ipykernel_53311/3720356303.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_pivoted['total_watch_duration'] = result_pivoted['duration'].apply(sum)


In [10]:
result_pivoted.head(10)

level_1,username,id,duration,e_time,s_time,type,total_watch_duration
4,4848720da58b52312b0340e836eb8e928474aa8d,c3403b4bddfb40759d6dc0d630a75484,[0.8847640833333333],[2021-12-27T07:16:19.067708+00:00],[2021-12-27T07:15:25.981863+00:00],video,0.884764
5,4848720da58b52312b0340e836eb8e928474aa8d,eca85b3f2d39415d86cc7aa1130cf96b,[3.7756333333333334],[2021-12-27T07:20:05.605708+00:00],[2021-12-27T07:16:19.067708+00:00],video,3.775633
9,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,a511e497e3a14e50b7e44aca1f726ebe,[14.019456566666666],[2021-12-23T13:14:45.884003+00:00],[2021-12-23T13:00:44.716609+00:00],video,14.019457
10,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,c3403b4bddfb40759d6dc0d630a75484,[1.58707355],[2021-12-23T12:53:54.625610+00:00],[2021-12-23T12:52:19.401197+00:00],video,1.587074
11,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,eca85b3f2d39415d86cc7aa1130cf96b,"[6.834849983333333, 5.904826283333334]","[2021-12-23T13:00:44.716609+00:00, 2021-12-23T...","[2021-12-23T12:53:54.625610+00:00, 2021-12-23T...",video,12.739676


## 7. Obtain Video watch percentage

In [11]:
import os
# Read video duration file
if os.path.exists(VIDEO_DURATION_FILE_PATH):
    print(f"File found: {VIDEO_DURATION_FILE_PATH}")
else:
    print(f"File not found at: {VIDEO_DURATION_FILE_PATH}")
duration_df = pd.read_excel(VIDEO_DURATION_FILE_PATH)
duration_df = duration_df.rename(columns={'duration': 'video_duration'})
print(duration_df)

File found: course_data/ARCH301/ARCH301_video_duration.xlsx
                            url_name  video_duration
0   002d0e3d0fde4e1a95fad68d0ce9ac46         1.03075
1   c0ded30db49f42a5892f33ca4c41d541         2.07182
2   eca85b3f2d39415d86cc7aa1130cf96b         3.53777
3   d5a96be443f6413f95ee4f393119b33d         2.63323
4   7c8fa71cee1043fa8ce7a67c9034610b         0.87395
5   83cdbca89d1f4740af19ba7ec9cb3abf         3.47875
6   0b8d8b0c55ee444896418403879ef90b         0.53867
7   9a0735d2d1c34204a53efd6337294133         0.63963
8   ae726b774bda45bfb157294d6c9f30e2         0.54507
9   574ca342e4bc4b67bff95c3abbd1cc5d         0.47047
10  05e1f000fb914c9aa3e966a0b4dd001a         3.03003
11  90549c8967954eefb138b939af88b26a         2.39928
12  eae241bfdf304638ad99e978d9fa72b1         0.57670
13  7ecd6cca41c744aebabde06e6b074e42         3.15662
14  ded64a52712545408f10fbd076b0ed64         3.37030
15  1ec479a9d8a342768e94000ae0bc5a28         0.72640
16  7b3f8261d61545cebffd760d6017007c   

In [12]:
merged_watch_df = pd.merge(result_pivoted, duration_df, left_on='id', right_on='url_name', how='left')
merged_watch_df['watch_percentage'] = merged_watch_df['total_watch_duration'] / merged_watch_df['video_duration'] 

merged_watch_df.head(10)

,username,id,duration,e_time,s_time,type,total_watch_duration,url_name,video_duration,watch_percentage
0,4848720da58b52312b0340e836eb8e928474aa8d,c3403b4bddfb40759d6dc0d630a75484,[0.8847640833333333],[2021-12-27T07:16:19.067708+00:00],[2021-12-27T07:15:25.981863+00:00],video,0.884764,c3403b4bddfb40759d6dc0d630a75484,0.54363,1.627512
1,4848720da58b52312b0340e836eb8e928474aa8d,eca85b3f2d39415d86cc7aa1130cf96b,[3.7756333333333334],[2021-12-27T07:20:05.605708+00:00],[2021-12-27T07:16:19.067708+00:00],video,3.775633,eca85b3f2d39415d86cc7aa1130cf96b,3.53777,1.067235
2,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,a511e497e3a14e50b7e44aca1f726ebe,[14.019456566666666],[2021-12-23T13:14:45.884003+00:00],[2021-12-23T13:00:44.716609+00:00],video,14.019457,a511e497e3a14e50b7e44aca1f726ebe,2.97243,4.716497
3,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,c3403b4bddfb40759d6dc0d630a75484,[1.58707355],[2021-12-23T12:53:54.625610+00:00],[2021-12-23T12:52:19.401197+00:00],video,1.587074,c3403b4bddfb40759d6dc0d630a75484,0.54363,2.919400
4,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,eca85b3f2d39415d86cc7aa1130cf96b,"[6.834849983333333, 5.904826283333334]","[2021-12-23T13:00:44.716609+00:00, 2021-12-23T...","[2021-12-23T12:53:54.625610+00:00, 2021-12-23T...",video,12.739676,eca85b3f2d39415d86cc7aa1130cf96b,3.53777,3.601047


## 8. Calculate average video watch percentage for each user

In [13]:
# Compute the average watch percentage per user
avg_watch_df = merged_watch_df.groupby("username", as_index=False)["watch_percentage"].mean()

avg_watch_df.head(10)

,username,watch_percentage
0,4848720da58b52312b0340e836eb8e928474aa8d,1.347373
1,ba36271878fa5e5d3eeae4bb2efc2d39ebccf436,3.745648


## 9. Exclude outlier watch percentage

In [14]:
import pandas as pd

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = avg_watch_df['watch_percentage'].quantile(0.25)
Q3 = avg_watch_df['watch_percentage'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define bounds for filtering outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to keep only rows within the IQR bounds
filtered_df = avg_watch_df[(avg_watch_df['watch_percentage'] >= lower_bound) & 
                           (avg_watch_df['watch_percentage'] <= upper_bound)]


In [16]:
filtered_df.to_csv(WATCH_PERCENTAGE_FILE_PATH, index=False)

## 9. Visualize distribution

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define the custom bin edges (0 to 5 with intervals of 0.25)
#bins = np.arange(0, 5.25, 0.25)

# Create a figure and plot
fig, ax = plt.subplots(figsize=(10, 5))
sns.ecdfplot(data=filtered_df, x="watch_percentage", ax=ax, color="royalblue", linewidth=4)

# Add labels
ax.set_xlabel("Average Watch Percentage Per User")
ax.set_ylabel("Cumulative Probability")
#ax.set_title(GRAPH_TITLE)

# Adjust axes
ax.set_xlim(left=0)
ax.set_ylim(bottom=0, top=1.05)  # Since CDF values go from 0 to 1

# Adjust spines to intersect at (0, 0)
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))